In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

<h3> Step 0: Data Sanitation </h3>
<li> null values (Nan), duplicates, object -> float/int</li>

In [ ]:
df = df.copy()
df.isnull().sum() #why is it missing, is it random? : input (impute) or delete? (some decisions come later outlier analysis, but some can be taken now)
df.duplicated().sum()
# dupli_rows = df[df.duplicated()]
# df.drop.duplicates() 
# check for special characters ??/>, delete and convert to int/float etc.

<h3>Step 1: Simply Look at the Data and state what you already can see</h3>

In [ ]:
df.type
df.describe()  #mean,sd, min,max

In [ ]:
#jointplot with distribution and regression line
sns.jointplot(data=df, x="alcohol", y="lifespan",marginal_kws=dict(bins=35), kind='reg')

<li>Decision: imputing do we detect strange outliers in the data do we impute or delete?</li>

<li> Quick Overall Graphical Overview (!warning, takes ~2min or more)</li>

In [ ]:
g = sns.PairGrid(df)
g.map(sns.lineplot)

<h3> Step 2: Zoom in on individual features </h3>
<li> Outliers </li>
<li> Strange data points </li>
<li> look for correlations: linear/log/etc.</li>
<li> Is the feature normally distributed </li>

In [ ]:
#Show scatter, boxplot a
fig, axes = plt.subplots(1,2,figsize=(15,5))
sns.stripplot(df, x='alcohol', y='lifespan',size=1, color=".8",ax=axes[0])
sns.boxplot(df[['alcohol']],ax=axes[1])

In [ ]:
#Show lineplot and/or jointplot, check for correlation (linear, positive/negative)
sns.lineplot(df,x='alcohol',y='lifespan')

In [ ]:
sns.jointplot(df, x="alcohol", y="lifespan",marginal_kws=dict(bins=35), kind='reg')

In [ ]:
#Show Q-Q plot and draw conclusion on linearity and if linear regression is applicable 
from seaborn_qqplot import pplot
myplot = pplot(df, x="lifespan", y="alcohol", kind='qq', height=5)

<h3> Step 4: Correlations of independent variables with dependent variable lifespan</h3>

__[Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)__

$ \rho_{x,y} = \frac{cov(x,y)}{\sigma_{x}\sigma{y}}$



In [ ]:
#colorfull matrix, showing correlations
df_corr = df[['lifespan', 'genetic', 
      #'length', 'mass', 
      'exercise', 'smoking',
       'alcohol', 'sugar', 'BMI']].dropna().corr()
df_corr.style.background_gradient(cmap='RdBu')

In [ ]:
#graphical correlation matrix view
sns.heatmap(df_corr,annot=True)

<li>Calculate P-values</li>

If p<0.05 (almost 0) the correlation for the feature is extremely likely to happen again if we collect more sample data and thus representative for the entire set


In [ ]:
r,p = stats.pearsonr(df.lifespan,df.smoking)
print('smoking corr:',round(r,4))
print('smoking p-val:',round(p,4))
r,p = stats.pearsonr(df.lifespan,df.exercise)
print('exercise corr:',round(r,4))
print('exercise p-val:',round(p,4))
r,p = stats.pearsonr(df.lifespan,df.genetic) #capital sensitive .<columnname>
print('BMI corr:',round(r,4))
print('BMI p-val:',round(p,4))

<h3>Step 5: If correlated, linear-ish graph vs lifespan, normal-ish distributed and continuous in R then we can apply linear regression</h3>
- Inputs: Smoking, Exercise<br>
- Output: Lifespan<br>

$ y=\alpha*x_{smoking}+\beta*x_{exercise}+c $

In [ ]:
from sklearn import linear_model 
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=0)

X = train[['smoking', 'exercise']]
y = train.lifespan
regr = linear_model.LinearRegression()
regr.fit(X, y) 

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)
print(f'c would be:',regr.predict([[0,0]]))

In regression, the $R^{2}$ coefficient of determination is a statistical measure of how well the regression predictions approximate the real data points.  <br>
Close to 1 means there is a strong correlation between the independent variables smoking, exercise and the dependent variable lifespan.

In [ ]:
print(train.shape, test.shape)
score = regr.score(test[['smoking', 'exercise']],test.lifespan)
print(f'coefficient of determination(R\N{SUPERSCRIPT TWO}):', score)

<h3>Step 6: Calculate the PremiumFactor for the insurance </h3><br>

$ PremiumFactor=\frac{genetic}{lifespan} $